In [1]:
import pyodbc
import pandas as pd
import json
import requests
import re
from os import listdir
from os.path import isfile, join

import glob
import os
#for OSX
import jaydebeapi
import subprocess
from jinja2 import Template
import pandas as pd
from pathlib import Path
import numpy as np

In [42]:
db_path = "../CITS20220208AH.mdb"

#/Users/joaoalmeida
ucanaccess_jars = [
    "/Users/joaoalmeida/Downloads/UCanAccess-5.0.1.bin/ucanaccess-5.0.1.jar",
    "/Users/joaoalmeida/Downloads/UCanAccess-5.0.1.bin/lib/commons-lang3-3.8.1.jar",
    "/Users/joaoalmeida/Downloads/UCanAccess-5.0.1.bin/lib/commons-logging-1.2.jar",
    "/Users/joaoalmeida/Downloads/UCanAccess-5.0.1.bin/lib/hsqldb-2.5.0.jar",
    "/Users/joaoalmeida/Downloads/UCanAccess-5.0.1.bin/lib/jackcess-3.0.1.jar",
]
classpath = ":".join(ucanaccess_jars)
cnxn = jaydebeapi.connect(
    "net.ucanaccess.jdbc.UcanaccessDriver",
    f"jdbc:ucanaccess://{db_path};newDatabaseVersion=V2010",
    ["", ""],
    classpath
    )
crsr = cnxn.cursor()

# Create Package Info
### Treat it and go downwards from here

In [43]:
#drugs for starting
list_meds=["Amlodipina + Candesartan","Ramipril + Amlodipina","Oximetazolina","Fumarato de dimetilo","Irbesartan","Carbonato de cálcio + Colecalciferol","Drospirenona + Etinilestradiol","Atorvastatina + Perindopril + Amlodipina","Trastuzumab","Nitroglicerina","Insulina glargina","Atorvastatina","Teriparatida","Ciclosporina","Amlodipina","Cefuroxima","Salbutamol","Goserrelina","Tafluprost","Clotrimazol","Itraconazol","Hidrocortisona","Enoxaparina sódica","Sinvastatina","Tramadol","Lidocaína","Morfina","Metformina","Paracetamol","Diclofenac","Irbesartan","Fumarato de dimetilo","Oximetazolina","Ibuprofeno","Carbamazepina","Cloroquina","Imatinib","Diazepam","Omeprazol","Enalapril"]
len(list_meds)

40

In [44]:
produto_sql="""select * from REF_DCIPT where DESCR in ('{}')""".format("','".join(list_meds))
med_ids = pd.read_sql(produto_sql,cnxn)
med_ids

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,DCIPT_ID,DESCR,ABREV_CHNM,IND_ATIVO
0,23,Carbamazepina,None,S
1,31,Cefuroxima,None,S
2,42,Clotrimazol,None,S
3,45,Diazepam,None,S
4,46,Diclofenac,None,S
5,50,Enalapril,None,S
6,71,Ibuprofeno,None,S
7,78,Metformina,None,S
8,92,Omeprazol,None,S
9,95,Paracetamol,None,S


In [45]:
dcipt_ids=med_ids["DCIPT_ID"].values.tolist()
dcipt_ids=[str(d) for d in dcipt_ids]

In [46]:
package_medicinal_product_sql="""
               select e.*,m.Dosagem,m.Nome,m.ind_Generico,ff.descr AS form_farm_descr,ff.Forma_Farm_ID,te.DESCR as tipo_emb_descr,
                tp.DESCR as Tipo_Prod_descr, gh.Descr as Grupo_Hom_descr, ea.Descr as Estado_AIM_Descr, rcdp.Descr as Class_Disp_descr,
                ta.Descr as Titular_AIM_descr, 
                dci.descr AS DCIPT_descr, dci.DCIPT_ID 
               from (((((((((
                   Embalagem as e

                INNER join produto as m on m.prod_id=e.prod_ID)
                left join ref_Tipo_emb as te on te.Tipo_emb_ID=e.Tipo_emb_ID )
                left join ref_Tipo_produto as tp on tp.Tipo_Produto_ID=m.Tipo_Produto_ID)
                left join ref_Forma_Farm as ff on ff.Forma_Farm_ID=m.Forma_Farm_ID)
                left join ref_Grupo_Hom as gh on gh.Grupo_Hom_cod=e.Grupo_Hom_cod)
                left join ref_Estado_AIM as ea on ea.Estado_AIM_ID=m.Estado_AIM_ID)
                left join lnk_prod_CLASS_DISP as cd on cd.prod_id=m.prod_id)
                left join ref_CLASS_DISPENSA_PROD as rcdp on rcdp.class_disp_prod_id=cd.class_disp_prod_id)
                left join ref_Titular_AIM as ta on ta.Titular_AIM_ID=m.Titular_AIM_ID)
                left join ref_DCIPT as dci on dci.DCIPT_ID=m.DCIPT_ID
                where ea.Estado_AIM_ID=1  and dci.DCIPT_ID in ({})""".format(",".join(dcipt_ids))

package_medicinal_product = pd.read_sql(package_medicinal_product_sql,cnxn)
#package_medicinal_product.to_csv("test_package.csv")
#from this, go downwards
package_medicinal_product

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,EMB_ID,PROD_ID,NR_REGISTO,DESCR,GRUPO_HOM_COD,QUANT_TOT,UNID_QUANT_TOT_ID,QUANTIDADE,UNID_QUANT_ID,TIPO_EMB_ID,...,DESCR,FORMA_FARM_ID,DESCR,DESCR,DESCR,DESCR,DESCR,DESCR,DESCR,DCIPT_ID
0,384,212,9686105,Frasco conta-gotas - 1 unidade(s) - 10 ml,GH0000,1.0,32,10.0,12.0,NaN,...,"Colírio, solução",123,None,Medicamento Uso Humano,None,Autorizado,MNSRM,"Laboratório Edol - Produtos Farmacêuticos, S.A.",Oximetazolina,883
1,747,446,9936302,Blister - 20 unidade(s),GH0000,20.0,32,NaN,NaN,NaN,...,Comprimido,271,None,Medicamento Uso Humano,None,Autorizado,MNSRM,Sidefarma - Sociedade Industrial de Expansão F...,Paracetamol,95
2,748,446,9936310,Blister - 4 unidade(s),GH0000,4.0,32,NaN,NaN,NaN,...,Comprimido,271,None,Medicamento Uso Humano,None,Autorizado,MNSRM,Sidefarma - Sociedade Industrial de Expansão F...,Paracetamol,95
3,808,487,9517813,Ampola - 1 unidade(s) - 6 ml,GH0000,1.0,32,6.0,12.0,NaN,...,Pó e solvente para solução injetável,212,None,Medicamento Uso Humano,None,Autorizado,MSRM,"Laboratórios Atral, S.A.",Cefuroxima,31
4,809,488,9517821,Ampola - 1 unidade(s) - 15 ml,GH0000,1.0,32,15.0,12.0,NaN,...,Pó e solvente para solução injetável,212,None,Medicamento Uso Humano,None,Autorizado,MSRM,"Laboratórios Atral, S.A.",Cefuroxima,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5176,2185415,607383,5820071,Blister - 540 unidade(s),GH0000,540.0,32,NaN,NaN,NaN,...,Comprimido,271,None,Medicamento Uso Humano,None,Autorizado,MNSRM,"Generis Farmacêutica, S.A.",Paracetamol,95
5177,2185416,607383,5820105,Blister - 540 unidade(s),GH0000,540.0,32,NaN,NaN,NaN,...,Comprimido,271,None,Medicamento Uso Humano,None,Autorizado,MNSRM,"Generis Farmacêutica, S.A.",Paracetamol,95
5178,2191575,675104,5821814,Frasco para injetáveis - 1 unidade(s),GH0000,1.0,32,NaN,NaN,NaN,...,Pó para concentrado para solução para perfusão,565,None,Medicamento Uso Humano,None,Autorizado,MSRM restrita - Alínea a),"Accord Healthcare, S.L.U.",Trastuzumab,1107
5179,2201075,625585,5826839,Blister - 56 unidade(s),GH0000,56.0,32,NaN,NaN,NaN,...,Comprimido,271,None,Medicamento Uso Humano,None,Autorizado,MSRM,"Generis Farmacêutica, S.A.",Enalapril,50


In [47]:
#get-columns-name
columns_sql="""SELECT * FROM Embalagem limit 5"""
col_name=pd.read_sql(columns_sql,cnxn).columns.tolist()
col_name.extend(["Dosagem","Nome","ind_generico","form_farm_descr","Forma_farm_id","tipo_emb_descr","Tipo_prod_descr","Grupo_hom_descr","Estado_AIM_descr","Class_Disp_descr","Titular_AIM_descr","DCIPT_descr","DCIPT_ID"])
col_name
package_medicinal_product.columns=col_name

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [48]:
def create_package_name(row):
    return row["Nome"] +" "+row["Dosagem"]+" "+row["form_farm_descr"] + " " + row["DESCR"]

package_medicinal_product["full_name"]=package_medicinal_product.apply(create_package_name,axis=1)


In [49]:
def parse_strength(row):
    col=row["Dosagem"]
   # print(col)
    """
    cases:
    50mg
    50 mg
    0.5mg/ml
    5mg/10ml
    1 mg/ml
    1200 mg/6 ml + 800 U.I./6 ml
    """
    def parse_single_instance(col):
        if col.count("/")==0: #no denominator
        # print(parse_num(col))
            numerator_value,numerator_unit=parse_num(col)
            return numerator_value,numerator_unit,None,None
        elif col.count("/")==1:
            #print(col)
            num=col.split("/")[0]
            dem=col.split("/")[1]
       #     print(num,"   >->",dem)
            numerator_value,numerator_unit=parse_num(num)
            denominator_value,denominator_unit=parse_dem(dem)
           # print(numerator_value,numerator_unit,denominator_value,denominator_unit)
            return numerator_value,numerator_unit,denominator_value,denominator_unit
        else:
            raise ValueError(col)
    def parse_num(num):
        numerator_value_tag="\d+\.?\d{0,3}"
        numerator_unit_tag="\s(\D+)$"
        try:
            numerator_value=re.findall(numerator_value_tag,num)[0]
            numerator_unit=re.findall(numerator_unit_tag,num)[0]
        except:
            raise ValueError(num)
        return numerator_value,numerator_unit
    def parse_dem(dem):
        """
        2 ml
        g 
        0.5 ml
        0.5ml
        ml
        """
        denominator_unit_tag=re.compile("\s?(\D+)$")
        denominator_value_tag=re.compile("\d\.?\d?")
        #print(dem)
        try:
            if denominator_value_tag.match(dem): #if has number
                denominator_value=denominator_value_tag.findall(dem)[0]
            else: #defaults to 1
                denominator_value=1
            
            denominator_unit=denominator_unit_tag.findall(dem)[0]

        except:
            raise ValueError(dem)
        return denominator_value,denominator_unit
       # print(denominator_unit)


    if col.count("+")==0:

       return parse_single_instance(col)
    else:
     #   print(col)
        numerator_unit_list=[]
        denominator_unit_list=[]
        numerator_value_list=[]
        denominator_value_list=[]
        for prod in col.split("+"):
            numerator_value,numerator_unit,denominator_value,denominator_unit=parse_single_instance(prod)
            numerator_unit_list.append(numerator_unit)
            if denominator_unit:
                denominator_unit_list.append(denominator_unit)
            
            numerator_value_list.append(numerator_value)
            if denominator_value:
                denominator_value_list.append(denominator_value)
        if len(denominator_value_list)>0:

            return numerator_value_list,numerator_unit_list,denominator_value_list,denominator_unit_list
        else:   
            return numerator_value_list,numerator_unit_list,None,None

package_medicinal_product[["numerator_value","numerator_unit","denominator_value","denominator_unit"]]=package_medicinal_product.apply(parse_strength,axis=1,result_type='expand')


In [50]:
package_medicinal_product[["full_name","Dosagem","numerator_value","numerator_unit","denominator_value","denominator_unit"]].iloc[50:90,:]

,full_name,Dosagem,numerator_value,numerator_unit,denominator_value,denominator_unit
50,Dafalgan 500mg 500 mg Comprimido efervescente ...,500 mg,500,mg,None,None
51,Zoref 250 mg/5 ml Granulado para suspensão ora...,250 mg/5 ml,250,mg,5,ml
52,Fenil-V Dispersivel 46.5 mg Comprimido dispers...,46.5 mg,46.5,mg,None,None
53,Fenil-V 50 mg Comprimido gastrorresistente Bli...,50 mg,50,mg,None,None
54,Fenil-V 100 mg Supositório Blister - 12 unidad...,100 mg,100,mg,None,None
55,Fenil-V Retard 100 mg Cápsula de libertação pr...,100 mg,100,mg,None,None
56,Albosan 20 mg Cápsula gastrorresistente Frasco...,20 mg,20,mg,None,None
57,Albosan 20 mg Cápsula gastrorresistente Frasco...,20 mg,20,mg,None,None
58,Gino-Canesten 100 mg Comprimido vaginal Bliste...,100 mg,100,mg,None,None
59,Gino-Canesten 10 mg/g Creme vaginal Bisnaga - ...,10 mg/g,10,mg,1,g


In [51]:
def transform_dcipt_to_list(row):
    """ transform combinations to strucutred
    """
    if "+" in row["DCIPT_descr"]:
        result=row["DCIPT_descr"].split("+")
        #print(row["DCIPT_descr"])
        return [prod.strip() for prod in result]
    return row["DCIPT_descr"]
package_medicinal_product["DCIPT_descr_parse"]=package_medicinal_product.apply(transform_dcipt_to_list,axis=1)


In [52]:
package_medicinal_product.head()

,EMB_ID,PROD_ID,NR_REGISTO,DESCR,GRUPO_HOM_COD,QUANT_TOT,UNID_QUANT_TOT_ID,QUANTIDADE,UNID_QUANT_ID,TIPO_EMB_ID,...,Class_Disp_descr,Titular_AIM_descr,DCIPT_descr,DCIPT_ID,full_name,numerator_value,numerator_unit,denominator_value,denominator_unit,DCIPT_descr_parse
0,384,212,9686105,Frasco conta-gotas - 1 unidade(s) - 10 ml,GH0000,1.0,32,10.0,12.0,NaN,...,MNSRM,"Laboratório Edol - Produtos Farmacêuticos, S.A.",Oximetazolina,883,"Alerjon 0.25 mg/ml Colírio, solução Frasco con...",0.25,mg,1,ml,Oximetazolina
1,747,446,9936302,Blister - 20 unidade(s),GH0000,20.0,32,NaN,NaN,NaN,...,MNSRM,Sidefarma - Sociedade Industrial de Expansão F...,Paracetamol,95,Febridol 500 mg Comprimido Blister - 20 unidad...,500,mg,None,None,Paracetamol
2,748,446,9936310,Blister - 4 unidade(s),GH0000,4.0,32,NaN,NaN,NaN,...,MNSRM,Sidefarma - Sociedade Industrial de Expansão F...,Paracetamol,95,Febridol 500 mg Comprimido Blister - 4 unidade(s),500,mg,None,None,Paracetamol
3,808,487,9517813,Ampola - 1 unidade(s) - 6 ml,GH0000,1.0,32,6.0,12.0,NaN,...,MSRM,"Laboratórios Atral, S.A.",Cefuroxima,31,Antibioxime 750 mg/6 ml Pó e solvente para sol...,750,mg,6,ml,Cefuroxima
4,809,488,9517821,Ampola - 1 unidade(s) - 15 ml,GH0000,1.0,32,15.0,12.0,NaN,...,MSRM,"Laboratórios Atral, S.A.",Cefuroxima,31,Antibioxime 1500 mg/15 ml Pó e solvente para s...,1500,mg,15,ml,Cefuroxima


In [53]:
def transform_list_to_string(x):
    if type(x)==list:
        return ";".join(x)
    return x
package_medicinal_product["numerator_value"]=package_medicinal_product["numerator_value"].apply(transform_list_to_string)
package_medicinal_product["numerator_unit"]=package_medicinal_product["numerator_unit"].apply(transform_list_to_string)
package_medicinal_product["denominator_value"]=package_medicinal_product["denominator_value"].apply(transform_list_to_string)
package_medicinal_product["denominator_unit"]=package_medicinal_product["denominator_unit"].apply(transform_list_to_string)
package_medicinal_product["DCIPT_descr_parse"]=package_medicinal_product["DCIPT_descr_parse"].apply(transform_list_to_string)


In [54]:
routes_sql="""select lpva.PROD_ID,lpva.VIA_ADMIN_ID,rva.DESCR from lnk_prod_via_admin lpva 
left join ref_via_admin rva on lpva.via_admin_id=rva.via_admin_id
  """
routes = pd.read_sql(routes_sql,cnxn)
routes

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,PROD_ID,VIA_ADMIN_ID,DESCR
0,50803,48,Via oral
1,623263,48,Via oral
2,1593,48,Via oral
3,49252,48,Via oral
4,623622,48,Via oral
...,...,...,...
20110,656665,48,Via oral
20111,652044,48,Via oral
20112,652045,48,Via oral
20113,653665,42,Via intravenosa


In [55]:
def concatenate_values_from_list(x):
   # print(type(x))
   # print(x.values)
    #print(";".join([str(d) for d in x.values]))
    return ";".join([str(d) for d in x.values])


In [56]:
# get CPARM and mapp to infarmed
cparm_routes=pd.read_excel("CPARM_V1.0.xlsx","Routes of Administration")
cparm_routes["mapping_term"]=cparm_routes["Preferred Term (PT) PT | Termo de Preferência PT"]
cparm_routes.loc[cparm_routes["Code EDQM |Código EDQM"]==20020000,"mapping_term"]="Via inalatória"
cparm_routes.loc[cparm_routes["Code EDQM |Código EDQM"]==20047000,"mapping_term"]="Via intravítreo"
cparm_routes.loc[cparm_routes["Code EDQM |Código EDQM"]==20025500,"mapping_term"]="Uso intracameral"
cparm_routes.loc[cparm_routes["Code EDQM |Código EDQM"]==20020000,"mapping_term"]="Via inalatória"


In [57]:
mapp_routes_edqm=routes.merge(cparm_routes,how="left",left_on="DESCR",right_on="mapping_term")
mapp_routes_edqm[mapp_routes_edqm["Code EDQM |Código EDQM"].isnull()]
#Uso epicutâneo	


,PROD_ID,VIA_ADMIN_ID,DESCR,Code EDQM |Código EDQM,Fully Specified Name (FSN) EN | Termo Totalmente Especificado EN,Definition EN | Definição EN,Preferred Term (PT) PT | Termo de Preferência PT,Definition PT | Definição PT,Domain | Âmbito,Version | Versão,Type of Change | Tipo Alteração,mapping_term
2814,40745,124,Uso epicutâneo,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
9793,45241,124,Uso epicutâneo,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
13896,40746,124,Uso epicutâneo,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN


In [58]:
routes_agg=mapp_routes_edqm.groupby("PROD_ID").aggregate({"VIA_ADMIN_ID":concatenate_values_from_list,"DESCR":concatenate_values_from_list,"Code EDQM |Código EDQM":concatenate_values_from_list,"Fully Specified Name (FSN) EN | Termo Totalmente Especificado EN":concatenate_values_from_list}).reset_index()
routes_agg.columns=["PROD_ID","routes_id","routes_descr","edqm_routes_id","edqm_routes_descr_en"]

In [59]:
product_with_routes=package_medicinal_product.merge(routes_agg,on="PROD_ID",how="left")

In [60]:
atc_sql="""select lpa.PROD_ID,lpa.ATC_COD,fa.DESCR from lnk_prod_atc lpa
left join ref_atc fa on lpa.atc_cod=fa.atc_cod
  """
atc = pd.read_sql(atc_sql,cnxn)
atc

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,PROD_ID,ATC_COD,DESCR
0,600228,N03AX14,levetiracetam
1,52417,N03AX14,levetiracetam
2,52265,N03AX14,levetiracetam
3,54829,J01XA01,vancomycin
4,35943,A16AX05,zinc acetate
...,...,...,...
18910,608082,H01BA04,terlipressin
18911,42315,N05CD08,midazolam
18912,29767,N02BA01,acetylsalicylic acid
18913,584341,N05CD08,midazolam


In [61]:
atc_agg=atc.groupby("PROD_ID").aggregate({"ATC_COD":concatenate_values_from_list,"DESCR":concatenate_values_from_list}).reset_index()
atc_agg.columns=["PROD_ID","atc_code","atc_descr"]

KeyboardInterrupt: 

In [ ]:
product_with_routes_atc=product_with_routes.merge(atc_agg,on="PROD_ID",how="left")

In [ ]:
cparm_pharm_forms=pd.read_excel("CPARM_V1.0.xlsx","Pharmaceutical Dose Forms")
#remove duplicates terms (tablet for 12200 and 102190000)
cparm_pharm_forms["mapping_term"]=cparm_pharm_forms["Preferred Term (PT) PT | Termo de Preferência PT"]
cparm_pharm_forms["mapping_term"]=cparm_pharm_forms["mapping_term"].apply(lambda x: x.strip())
cparm_pharm_forms.loc[cparm_pharm_forms["Code EDQM |Código EDQM"]==11005000,"mapping_term"]="Solução retal"
cparm_pharm_forms=cparm_pharm_forms[cparm_pharm_forms["Definition EN | Definição EN"]!=" - "]


In [ ]:
#for all (NOT DONE)
dose_forms_sql="""select * from ref_Forma_Farm """
dose_forms = pd.read_sql(dose_forms_sql,cnxn)
dose_forms

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,FORMA_FARM_ID,DESCR,ABREV,ABREV_CHNM,IND_ATIVO
0,0,N.A.,None,None,S
1,1,"Gotas orais, solução","Gotas orais, sol.",Sol oral,S
2,2,"Gotas orais, suspensão","Gotas orais, susp.",Susp oral,S
3,3,"Gotas orais, emulsão","Gotas orais, emul.",Emul oral,S
4,4,Solução oral,Sol. oral,Sol oral,S
...,...,...,...,...,...
510,1000,Solução para perfusão em cartucho,None,Sol inj,S
511,1001,Gel oftálmico em recipiente unidose,None,Gel oft,S
512,1002,Película sublingual,None,None,S
513,1003,"Pó, dispersão e veículo para concentrado para ...",None,None,S


In [ ]:
selected_dose_forms=product_with_routes_atc[["form_farm_descr","Forma_farm_id"]]
selected_dose_forms.drop_duplicates(inplace=True)
selected_dose_forms

/var/folders/8g/rym67m5d29dcjm2td47sgb_m0000gn/T/ipykernel_28779/1619014294.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_dose_forms.drop_duplicates(inplace=True)


,form_farm_descr,Forma_farm_id
0,"Colírio, solução",123
1,Comprimido,271
3,Pó e solvente para solução injetável,212
15,Supositório,186
19,Xarope,13
...,...,...
4373,Pó para solução oral em saqueta,950
4532,Pó para solução injetável ou para perfusão,537
4571,Suspensão oral em saqueta,983
4806,Gel intrauterino,997


In [ ]:
mapp_dose_form_edqm=selected_dose_forms.merge(cparm_pharm_forms,how="left",left_on="form_farm_descr",right_on="mapping_term")
mapp_dose_form_edqm[mapp_dose_form_edqm["Code EDQM |Código EDQM"].isnull()]
# Granulado orodispersível
# Sistema transdérmico	
# Solução injetável ou para perfusão

,form_farm_descr,Forma_farm_id,Code EDQM |Código EDQM,Fully Specified Name (FSN) EN | Termo Totalmente Especificado EN,Definition EN | Definição EN,Preferred Term (PT) PT | Termo de Preferência PT,Definition PT | Definição PT,Domain | Âmbito,Version | Versão,Type of Change | Tipo Alteração,mapping_term
28,Sistema transdérmico,98,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
60,Granulado orodispersível,952,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
65,Solução injetável ou para perfusão,548,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN


In [ ]:
cparm_pharm_forms=cparm_pharm_forms[["Code EDQM |Código EDQM","Fully Specified Name (FSN) EN | Termo Totalmente Especificado EN","Preferred Term (PT) PT | Termo de Preferência PT","mapping_term"]]
cparm_pharm_forms.columns=["edqm_dose_form_id","edqm_dose_form_descr_en","edqm_dose_form_descr_pt","mapping_term"]

In [ ]:
#add to final dataframe
cparm_pharm_forms
product_with_routes_atc_edqm_dose=product_with_routes_atc.merge(cparm_pharm_forms,how="left",left_on="form_farm_descr",right_on="mapping_term")

In [ ]:
product_with_routes_atc_edqm_dose

,EMB_ID,PROD_ID,NR_REGISTO,DESCR,GRUPO_HOM_COD,QUANT_TOT,UNID_QUANT_TOT_ID,QUANTIDADE,UNID_QUANT_ID,TIPO_EMB_ID,...,routes_id,routes_descr,edqm_routes_id,edqm_routes_descr_en,atc_code,atc_descr,edqm_dose_form_id,edqm_dose_form_descr_en,edqm_dose_form_descr_pt,mapping_term
0,384,212,9686105,Frasco conta-gotas - 1 unidade(s) - 10 ml,GH0000,1.0,32,10.0,12.0,NaN,...,47,Uso oftálmico,20051000.0,Ocular use,S01GA04,oxymetazoline,10604000.0,"Eye drops, solution","Colírio, solução","Colírio, solução"
1,747,446,9936302,Blister - 20 unidade(s),GH0000,20.0,32,NaN,NaN,NaN,...,48,Via oral,20053000.0,Oral use,N02BE01,paracetamol,10219000.0,Tablet,Comprimido,Comprimido
2,748,446,9936310,Blister - 4 unidade(s),GH0000,4.0,32,NaN,NaN,NaN,...,48,Via oral,20053000.0,Oral use,N02BE01,paracetamol,10219000.0,Tablet,Comprimido,Comprimido
3,808,487,9517813,Ampola - 1 unidade(s) - 6 ml,GH0000,1.0,32,6.0,12.0,NaN,...,42;30,Via intravenosa;Via intramuscular,20045000.0;20035000.0,Intravenous use;Intramuscular use,J01DC02,cefuroxime,11207000.0,Powder and solvent for solution for injection,Pó e solvente para solução injetável,Pó e solvente para solução injetável
4,809,488,9517821,Ampola - 1 unidade(s) - 15 ml,GH0000,1.0,32,15.0,12.0,NaN,...,42,Via intravenosa,20045000.0,Intravenous use,J01DC02,cefuroxime,11207000.0,Powder and solvent for solution for injection,Pó e solvente para solução injetável,Pó e solvente para solução injetável
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5176,2185415,607383,5820071,Blister - 540 unidade(s),GH0000,540.0,32,NaN,NaN,NaN,...,48,Via oral,20053000.0,Oral use,N02BE01,paracetamol,10219000.0,Tablet,Comprimido,Comprimido
5177,2185416,607383,5820105,Blister - 540 unidade(s),GH0000,540.0,32,NaN,NaN,NaN,...,48,Via oral,20053000.0,Oral use,N02BE01,paracetamol,10219000.0,Tablet,Comprimido,Comprimido
5178,2191575,675104,5821814,Frasco para injetáveis - 1 unidade(s),GH0000,1.0,32,NaN,NaN,NaN,...,42,Via intravenosa,20045000.0,Intravenous use,L01XC03,trastuzumab,50043000.0,Powder for concentrate for solution for infusion,Pó para concentrado para solução para perfusão,Pó para concentrado para solução para perfusão
5179,2201075,625585,5826839,Blister - 56 unidade(s),GH0000,56.0,32,NaN,NaN,NaN,...,48,Via oral,20053000.0,Oral use,C09AA02,enalapril,10219000.0,Tablet,Comprimido,Comprimido


In [ ]:
#testing with Amlor 10mg cápsulas caixa 100 - nao existe pt
#linha 137 - norvasc 10 mg caixa 30

In [ ]:
#check for duplicates
product_with_routes_atc_edqm_dose[product_with_routes_atc_edqm_dose.duplicated(['EMB_ID'], keep=False)]

,EMB_ID,PROD_ID,NR_REGISTO,DESCR,GRUPO_HOM_COD,QUANT_TOT,UNID_QUANT_TOT_ID,QUANTIDADE,UNID_QUANT_ID,TIPO_EMB_ID,...,routes_id,routes_descr,edqm_routes_id,edqm_routes_descr_en,atc_code,atc_descr,edqm_dose_form_id,edqm_dose_form_descr_en,edqm_dose_form_descr_pt,mapping_term


In [ ]:
product_with_routes_atc_edqm_dose.to_csv("infarmed_export_v0.csv")


In [ ]:
with open('template-packaged-medicinal-product R5.fsh', 'r') as file:
    templateString = file.read()
t = Template(templateString,trim_blocks=True)

DATA_FILE='infarmed_export_v0_amlodipine.csv'
#DATA_FILE='infarmed_export_v0.csv'
df=pd.read_csv(DATA_FILE,index_col=0)
df=df.astype(str)

#Path("../input/fsh/generated").mkdir(parents=True, exist_ok=True)
#t.stream(data=df.iloc[0:2]).dump('input/fsh/be-ampp.fsh')
t.stream(data=df).dump('../input/fsh/pt-pmp_r5.fsh')

UndefinedError: 'pandas.core.series.Series object' has no attribute 'DCIPT_id_parse'

In [ ]:
# writing to file
file1 = open('../input/fsh/pt-pmp.fsh', 'r')
Lines = file1.readlines()
  
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    if "// ERROR" in line:
        print("{} @{}".format( line.strip(),count))

FileNotFoundError: [Errno 2] No such file or directory: '../input/fsh/pt-pmp.fsh'

,EMB_ID,PROD_ID,NR_REGISTO,DESCR,GRUPO_HOM_COD,QUANT_TOT,UNID_QUANT_TOT_ID,QUANTIDADE,UNID_QUANT_ID,TIPO_EMB_ID,...,routes_id,routes_descr,edqm_routes_id,edqm_routes_descr_en,atc_code,atc_descr,edqm_dose_form_id,edqm_dose_form_descr_en,edqm_dose_form_descr_pt,mapping_term
5027,2157026,651369,5804133,Seringa pré-cheia - 20 unidade(s) - 0.6 ml,GH0000,20.0,32,0.6,12.0,NaN,...,42;58;144,Via intravenosa;Via subcutânea;Uso extracorpóreo,20045000.0;20066000.0;20011500.0,Intravenous use;Subcutaneous use;Extracorporea...,B01AB05,enoxaparin,50060300.0,Solution for injection in pre-filled syringe,Solução injetável em seringa pré-cheia,Solução injetável em seringa pré-cheia


In [ ]:
for index,row in product_with_routes_atc_edqm_dose.iterrows():
    if row["edqm_dose_form_id"]==np.nan:
        print(row["EMB_ID"])


In [ ]:
tt=product_with_routes_atc_edqm_dose["DCIPT_descr_parse"].unique()
list_unparse=[]
for ttt in tt:
    if ";" in ttt:
        for pl in ttt.split(";"):
            list_unparse.append(pl)
    else:
        list_unparse.append(ttt)

In [ ]:
item_dict_parse= {k:v for k,v in zip(list_unparse,list(range(1,len(list_unparse)+1)))}
item_dict_parse


{'Oximetazolina': 1,
 'Paracetamol': 2,
 'Cefuroxima': 3,
 'Irbesartan': 4,
 'Diazepam': 5,
 'Ibuprofeno': 6,
 'Clotrimazol': 7,
 'Diclofenac': 8,
 'Enalapril': 9,
 'Morfina': 10,
 'Omeprazol': 11,
 'Metformina': 12,
 'Nitroglicerina': 13,
 'Hidrocortisona': 14,
 'Carbonato de cálcio': 15,
 'Colecalciferol': 16,
 'Lidocaína': 17,
 'Enoxaparina sódica': 18,
 'Amlodipina': 43,
 'Tramadol': 20,
 'Salbutamol': 21,
 'Ciclosporina': 22,
 'Itraconazol': 23,
 'Carbamazepina': 24,
 'Atorvastatina': 39,
 'Sinvastatina': 26,
 'Goserrelina': 27,
 'Trastuzumab': 28,
 'Insulina glargina': 29,
 'Drospirenona': 30,
 'Etinilestradiol': 31,
 'Cloroquina': 32,
 'Imatinib': 33,
 'Teriparatida': 34,
 'Tafluprost': 35,
 'Candesartan': 37,
 'Fumarato de dimetilo': 38,
 'Perindopril': 40,
 'Ramipril': 42}

In [ ]:
def mapp_dci_pt_parse(x,map):
    if ";" in x:
        p=[]
        for r in x.split(";"):
            p.append(str(map[r]))
        return ";".join(p)
    else:
        return map[x]

In [ ]:
product_with_routes_atc_edqm_dose["DCIPT_id_parse"]=product_with_routes_atc_edqm_dose["DCIPT_descr_parse"].apply(mapp_dci_pt_parse,map=item_dict_parse)

In [62]:
product_with_routes_atc_edqm_dose[product_with_routes_atc_edqm_dose["DCIPT_ID"].isin([174,3446,3600,3633])].to_csv("infarmed_export_v0_amlodipine_multiple.csv")

In [63]:
# multiple elementsaa

with open('template-packaged-medicinal-product R5.fsh', 'r') as file:
    templateString = file.read()
t = Template(templateString,trim_blocks=True)

DATA_FILE='infarmed_export_v0_amlodipine_multiple.csv'
#DATA_FILE='infarmed_export_v0.csv'
df=pd.read_csv(DATA_FILE,index_col=0)
df=df.astype(str)

#Path("../input/fsh/generated").mkdir(parents=True, exist_ok=True)
#t.stream(data=df.iloc[0:2]).dump('input/fsh/be-ampp.fsh')
t.stream(data=df).dump('../input/fsh/pt-pmp_r5.fsh')

In [64]:
for k,v in item_dict_parse.items():
    print("#"+str(v)+" "+k)

#1 Oximetazolina
#2 Paracetamol
#3 Cefuroxima
#4 Irbesartan
#5 Diazepam
#6 Ibuprofeno
#7 Clotrimazol
#8 Diclofenac
#9 Enalapril
#10 Morfina
#11 Omeprazol
#12 Metformina
#13 Nitroglicerina
#14 Hidrocortisona
#15 Carbonato de cálcio
#16 Colecalciferol
#17 Lidocaína
#18 Enoxaparina sódica
#43 Amlodipina
#20 Tramadol
#21 Salbutamol
#22 Ciclosporina
#23 Itraconazol
#24 Carbamazepina
#39 Atorvastatina
#26 Sinvastatina
#27 Goserrelina
#28 Trastuzumab
#29 Insulina glargina
#30 Drospirenona
#31 Etinilestradiol
#32 Cloroquina
#33 Imatinib
#34 Teriparatida
#35 Tafluprost
#37 Candesartan
#38 Fumarato de dimetilo
#40 Perindopril
#42 Ramipril
